In [26]:
import pandas as pd
data = pd.read_csv('weatherAUS.csv')


In [27]:
data

Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
0       2008-12-01   Albury     13.4     22.9       0.6          NaN   
1       2008-12-02   Albury      7.4     25.1       0.0          NaN   
2       2008-12-03   Albury     12.9     25.7       0.0          NaN   
3       2008-12-04   Albury      9.2     28.0       0.0          NaN   
4       2008-12-05   Albury     17.5     32.3       1.0          NaN   
...            ...      ...      ...      ...       ...          ...   
145455  2017-06-21    Uluru      2.8     23.4       0.0          NaN   
145456  2017-06-22    Uluru      3.6     25.3       0.0          NaN   
145457  2017-06-23    Uluru      5.4     26.9       0.0          NaN   
145458  2017-06-24    Uluru      7.8     27.0       0.0          NaN   
145459  2017-06-25    Uluru     14.9      NaN       0.0          NaN   

        Sunshine WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  \
0            NaN           W           44.0          W  ...        71.0   
1            NaN         WNW           44.0        NNW  ...        44.0   
2            NaN         WSW           46.0          W  ...        38.0   
3            NaN          NE           24.0         SE  ...        45.0   
4            NaN           W           41.0        ENE  ...        82.0   
...          ...         ...            ...        ...  ...         ...   
145455       NaN           E           31.0         SE  ...        51.0   
145456       NaN         NNW           22.0         SE  ...        56.0   
145457       NaN           N           37.0         SE  ...        53.0   
145458       NaN          SE           28.0        SSE  ...        51.0   
145459       NaN         NaN            NaN        ESE  ...        62.0   

        Humidity3pm  Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  \
0              22.0       1007.7       1007.1       8.0       NaN     16.9   
1              25.0       1010.6       1007.8       NaN       NaN     17.2   
2              30.0       1007.6       1008.7       NaN       2.0     21.0   
3              16.0       1017.6       1012.8       NaN       NaN     18.1   
4              33.0       1010.8       1006.0       7.0       8.0     17.8   
...             ...          ...          ...       ...       ...      ...   
145455         24.0       1024.6       1020.3       NaN       NaN     10.1   
145456         21.0       1023.5       1019.1       NaN       NaN     10.9   
145457         24.0       1021.0       1016.8       NaN       NaN     12.5   
145458         24.0       1019.4       1016.5       3.0       2.0     15.1   
145459         36.0       1020.2       1017.9       8.0       8.0     15.0   

        Temp3pm  RainToday  RainTomorrow  
0          21.8         No            No  
1          24.3         No            No  
2          23.2         No            No  
3          26.5         No            No  
4          29.7         No            No  
...         ...        ...           ...  
145455     22.4         No            No  
145456     24.5         No            No  
145457     26.1         No            No  
145458     26.0         No            No  
145459     20.9         No           NaN  

[145460 rows x 23 columns]

In [28]:
data = data.drop(['Date', 'Location'], axis=1)


In [29]:
!pip install pybbn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import pandas as pd # for data manipulation 
import networkx as nx # for drawing graphs
import matplotlib.pyplot as plt # for drawing graphs

# for creating Bayesian Belief Networks (BBN)
from pybbn.graph.dag import Bbn
from pybbn.graph.edge import Edge, EdgeType
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.pptc.inferencecontroller import InferenceController

In [31]:
data['Temp9amCat'] = data['Temp9am'].apply(lambda x: 'cat1' if x <15 else 'cat2' if 15<=x<=26 else 'cat3')
data['Temp3pmCat'] = data['Temp3pm'].apply(lambda x: 'cat1' if x <24 else 'cat2' if 24<=x<=36 else 'cat3' )
data['WindSpeed9amCat'] = data['WindSpeed9am'].apply(lambda x: 'cat1' if x>16 else 'cat2')
data['WindSpeed3pmCat'] = data['WindSpeed3pm'].apply(lambda x: 'cat1' if x>15 else 'cat2')

In [32]:
data

MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine WindGustDir  \
0          13.4     22.9       0.6          NaN       NaN           W   
1           7.4     25.1       0.0          NaN       NaN         WNW   
2          12.9     25.7       0.0          NaN       NaN         WSW   
3           9.2     28.0       0.0          NaN       NaN          NE   
4          17.5     32.3       1.0          NaN       NaN           W   
...         ...      ...       ...          ...       ...         ...   
145455      2.8     23.4       0.0          NaN       NaN           E   
145456      3.6     25.3       0.0          NaN       NaN         NNW   
145457      5.4     26.9       0.0          NaN       NaN           N   
145458      7.8     27.0       0.0          NaN       NaN          SE   
145459     14.9      NaN       0.0          NaN       NaN         NaN   

        WindGustSpeed WindDir9am WindDir3pm  WindSpeed9am  ...  Cloud9am  \
0                44.0          W        WNW          20.0  ...       8.0   
1                44.0        NNW        WSW           4.0  ...       NaN   
2                46.0          W        WSW          19.0  ...       NaN   
3                24.0         SE          E          11.0  ...       NaN   
4                41.0        ENE         NW           7.0  ...       7.0   
...               ...        ...        ...           ...  ...       ...   
145455           31.0         SE        ENE          13.0  ...       NaN   
145456           22.0         SE          N          13.0  ...       NaN   
145457           37.0         SE        WNW           9.0  ...       NaN   
145458           28.0        SSE          N          13.0  ...       3.0   
145459            NaN        ESE        ESE          17.0  ...       8.0   

        Cloud3pm  Temp9am  Temp3pm  RainToday  RainTomorrow  Temp9amCat  \
0            NaN     16.9     21.8         No            No        cat2   
1            NaN     17.2     24.3         No            No        cat2   
2            2.0     21.0     23.2         No            No        cat2   
3            NaN     18.1     26.5         No            No        cat2   
4            8.0     17.8     29.7         No            No        cat2   
...          ...      ...      ...        ...           ...         ...   
145455       NaN     10.1     22.4         No            No        cat1   
145456       NaN     10.9     24.5         No            No        cat1   
145457       NaN     12.5     26.1         No            No        cat1   
145458       2.0     15.1     26.0         No            No        cat2   
145459       8.0     15.0     20.9         No           NaN        cat2   

        Temp3pmCat  WindSpeed9amCat WindSpeed3pmCat  
0             cat1             cat1            cat1  
1             cat2             cat2            cat1  
2             cat1             cat1            cat1  
3             cat2             cat2            cat2  
4             cat2             cat2            cat1  
...            ...              ...             ...  
145455        cat1             cat2            cat2  
145456        cat2             cat2            cat2  
145457        cat2             cat2            cat2  
145458        cat2             cat2            cat2  
145459        cat1             cat1            cat1  

[145460 rows x 25 columns]

In [33]:
def probs(data, child, parent1=None, parent2=None):
    if parent1==None:
        # Calculate probabilities
        prob=pd.crosstab(data[child], 'Empty', margins=False, normalize='columns').sort_index().to_numpy().reshape(-1).tolist()
    elif parent1!=None:
            # Check if child node has 1 parent or 2 parents
            if parent2==None:
                # Caclucate probabilities
                prob=pd.crosstab(data[parent1],data[child], margins=False, normalize='index').sort_index().to_numpy().reshape(-1).tolist()
            else:    
                # Caclucate probabilities
                prob=pd.crosstab([data[parent1],data[parent2]],data[child], margins=False, normalize='index').sort_index().to_numpy().reshape(-1).tolist()
    else: print("Error in Probability Frequency Calculations")
    return prob  

In [35]:
T9am = BbnNode(Variable(0,'Temp9am',['cat1','cat2','cat3']),probs(data,child = 'Temp3pmCat'))
T3pm = BbnNode(Variable(1,'Temp3pm',['cat1','cat2','cat3']),probs(data,parent1 = 'Temp9amCat',child = 'RainTomorrow'))
W9am = BbnNode(Variable(2,'WindSpeed9am',['cat1','cat2']),probs(data,child = 'WindSpeed3pmCat'))
W3pm = BbnNode(Variable(3,'WindSpeed3pm',['cat1','cat2']),probs(data,parent1 = 'WindSpeed9amCat',child = 'RainTomorrow'))
RT = BbnNode(Variable(4,'RainTomorrow',['No','Yes']),probs(data,child='RainTomorrow',parent1 = 'Temp3pmCat', parent2 = 'WindSpeed3pmCat' ))

In [37]:
bbn = Bbn() \
    .add_node(T9am) \
    .add_node(T3pm) \
    .add_node(W9am) \
    .add_node(W3pm) \
    .add_node(RT) \
    .add_edge(Edge(T9am, T3pm, EdgeType.DIRECTED)) \
    .add_edge(Edge(W9am, W3pm, EdgeType.DIRECTED)) \
    .add_edge(Edge(W3pm, RT, EdgeType.DIRECTED))\
    .add_edge(Edge(T3pm, RT, EdgeType.DIRECTED))

In [ ]:
join_tree = InferenceController.apply(bbn)

In [ ]:
pos = {0: (-1, 2), 1: (-1, 0.5), 2: (1, 0.5), 3: (0, -1)}

# Set options for graph looks
options = {
    "font_size": 16,
    "node_size": 4000,
    "node_color": "white",
    "edgecolors": "black",
    "edge_color": "red",
    "linewidths": 5,
    "width": 5,}
    
# Generate graph
n, d = bbn.to_nx_graph()
nx.draw(n, with_labels=True, labels=d, pos=pos, **options)

# Update margins and print the graph
ax = plt.gca()
ax.margins(0.10)
plt.axis("off")
plt.show()